In [1]:
!pwd

/home/dephinate/ASU/DL/MedicalChatBot


In [2]:
import os
from pathlib import Path

Import libraries    
we need the following:  
* something for document data
* something for splitting
* something for embeddings
* something for vectior indexing
* something for prompts
* something for llm

In [16]:
# Loader
import langchain
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
import pypdf
# Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings
# Vector Store
from langchain.vectorstores import Pinecone
import pinecone
# Prompts
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain



Setup Pinecone

In [4]:
from research import PINECONE_API_ENV, PINECONE_API_ENV

Load Data

In [7]:
# Extract data from the pdf
def load_pdf(path:Path):
    loader = DirectoryLoader(   # To load all pdfs from a directory
        path=path,
        glob="*.pdf",
        loader_cls=PyPDFLoader,
        show_progress=True
    )
    documents = loader.load()
    return documents


In [21]:
extracted_data = load_pdf("./data")


100%|██████████| 1/1 [00:09<00:00,  9.15s/it]


In [22]:
print(len(extracted_data))
print(type(extracted_data[0]))

759
<class 'langchain.schema.document.Document'>


Split Data

In [23]:
# parameters: chunk size and chunk overlap
def text_split(chunk_size:int, chunk_overlap:int, extracted_data:list[langchain.schema.document.Document]):
    splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size, chunk_overlap = chunk_overlap)
    chunks = splitter.split_documents(extracted_data)
    return chunks

In [24]:
chunks = text_split(chunk_size=500,chunk_overlap=20,extracted_data=extracted_data)

In [26]:
print(f"length of chunks: {len(chunks)}")

length of chunks: 6983


Create Embeddings

In [ ]:
# fucntion to download hugging face embeddings
def download_embeddings_from_huggingface(model_name:str):
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    return embeddings